In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load your dataset
#rescale=1./255
train_datagen = ImageDataGenerator(
    #rescale=1./255,            # Normalizing pixel values
    rotation_range=40,         # Randomly rotating images in the range (degrees, 0 to 180)
    width_shift_range=0.2,     # Randomly translating images horizontally
    height_shift_range=0.2,    # Randomly translating images vertically
    shear_range=0.2,           # Applying shear transformations
    zoom_range=0.2,            # Randomly zooming inside pictures
    horizontal_flip=True,      # Randomly flipping half of the images horizontally
    validation_split=0.2
)
train_generator = train_datagen.flow_from_directory(
        '/kaggle/input/face-classification-deep-learning/dataset/train',
        target_size=(224, 224),
        batch_size=512,
        class_mode='categorical',
        subset='training')
validation_generator = train_datagen.flow_from_directory(
        '/kaggle/input/face-classification-deep-learning/dataset/train',
        target_size=(224, 224),
        batch_size=512,
        class_mode='categorical',
        subset='validation')

# Define the model
base_model = tf.keras.applications.VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the convolutional base

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(7000, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

Found 112000 images belonging to 7000 classes.
Found 28000 images belonging to 7000 classes.
80134624/80134624 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 7000)              3591000   
                                                                 
Total params: 23,615,384
Trainable params: 3,591,000
Non-trainable params: 20,024,384
_________________________________________________________________


In [ ]:
history = model.fit(train_generator, validation_data=validation_generator, epochs=10)

Epoch 1/10
219/219 [==============================] - 2927s 13s/step - loss: 9.4375 - accuracy: 0.0049 - val_loss: 7.8549 - val_accuracy: 0.0211
Epoch 2/10
219/219 [==============================] - 1936s 9s/step - loss: 7.3572 - accuracy: 0.0404 - val_loss: 7.2394 - val_accuracy: 0.0465
Epoch 3/10
219/219 [==============================] - ETA: 0s - loss: 6.5057 - accuracy: 0.0833

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input

# Directory containing test images
test_dir = '/kaggle/input/face-classification-deep-learning/dataset/test'

# Initialize a DataFrame to store the results
results = []

# Loop over each image in the test directory
for img_name in os.listdir(test_dir):
    img_path = os.path.join(test_dir, img_name)
    
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    
    # Make a prediction
    predictions = model.predict(img_array)
    
    # Get the class with the highest probability
    predicted_class = np.argmax(predictions)
    
    # Append the result to the DataFrame
    results = results.append(predicted_class)
    
res = pd.DataFrame({"image_name":os.listdir(test_dir),"labels":results})

# Save the results to a CSV file
res.to_csv('/kaggle/working/predicted_labels.csv', index=False)